# import Libraries

In [4]:
import numpy as np

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

import eda_helper_functions

from sklearn.ensemble import IsolationForest

# Read the Training Data

In [5]:
file_path = r"C:\Users\lenovo i3\OneDrive\Documents\Project Data\Compus x project\filght-price-prediction\dataset\train.csv"

train = pd.read_csv(file_path)
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Delhi,Cochin,05:30:00,19:45:00,2295,2.0,In-flight meal not included,15864
1,Indigo,2019-05-06,Banglore,Delhi,13:00:00,15:50:00,170,0.0,No Info,3943
2,Multiple Carriers,2019-04-27,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
3,Jet Airways,2019-03-21,Delhi,Cochin,18:20:00,18:50:00,1470,2.0,No Info,12604
4,Air India,2019-03-09,Delhi,Cochin,08:00:00,19:15:00,675,1.0,No Info,16754
...,...,...,...,...,...,...,...,...,...,...
635,Air India,2019-03-01,Banglore,New Delhi,08:50:00,23:25:00,875,2.0,1 Short layover,26743
636,Jet Airways,2019-06-24,Delhi,Cochin,21:50:00,19:00:00,1270,1.0,In-flight meal not included,10262
637,Jet Airways,2019-03-06,Delhi,Cochin,19:45:00,19:45:00,1440,1.0,No Info,17024
638,Spicejet,2019-04-15,Kolkata,Banglore,17:10:00,19:40:00,150,0.0,No check-in baggage included,3841


## 2.1 Fixing Data Types

In [6]:
train.dtypes

airline             object
date_of_journey     object
source              object
destination         object
dep_time            object
arrival_time        object
duration             int64
total_stops        float64
additional_info     object
price                int64
dtype: object